In [0]:
# Importando módulo de seguimiento
import sys
import importlib
sys.path.append('/Workspace/Users/luisfdiaz@bcp.com.pe/ScoreGuardian')

import core.SegScore as rmm
importlib.reload(rmm)

# Librerias y fuciones generales
from pyspark.sql.functions import date_format, expr, to_date, date_sub, add_months, col, when, coalesce, trim, broadcast, avg, max, min, lit, concat, window, round as colround, upper, abs as sparkabs
from pyspark.sql import functions as F
from pyspark import StorageLevel
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType

#Mostrar los duplicados
def print_res(sparkf_df):
  import pandas as pd
  pd.set_option('display.max_rows', None)
  pd.set_option('display.max_columns', None)
  pd_spark_df = sparkf_df.toPandas()
 
  return pd_spark_df

In [0]:
# Importando base y configuraciones iniciales
base_seguimiento = spark.sql("select distinct * from catalog_lhcl_prod_bcp.bcp_edv_fabseg.t39290_hm_app_hipahlo_f")
base_seguimiento = base_seguimiento.withColumn("CODMES", col("CODMES").cast("integer"))
base_seguimiento = base_seguimiento.withColumn("DEF5", when(col("DISTANCIA_DEF24") <= 5, lit(1)).otherwise(lit(0)))
base_seguimiento = base_seguimiento.withColumn("DEF4", when(col("DISTANCIA_DEF24") <= 4, lit(1)).otherwise(lit(0))) 

base_seguimiento.persist(StorageLevel.MEMORY_AND_DISK).count()

# Verificando duplicandos en el segundo Join
dup = base_seguimiento.groupBy("CODCLAVEOPECTA", "CODMES").count().alias("count").filter(col("count")>1)
print_res(dup)

In [0]:
fc = pd.DataFrame([(1, 1), (1, 1), (1, 1)], columns=['CODMES', 'FAC_CAL_DRI_N'])
fm = pd.DataFrame([(1, 1), (1, 1), (1, 1)], columns=['CODMES', 'FAC_CAL_DRI_M'])

In [0]:
base_seguimiento = base_seguimiento.withColumn("ACT_HIP", when(col("CODMES")<202308, "ANTES").otherwise(col("ACT_HIP")))
base_seguimiento = base_seguimiento.dropDuplicates(subset=['CODMES', 'CODCLAVEOPECTA'])

In [0]:
pd_cal='PD_2024Q4vf'#vigente
pd_rbm='PD_RBM'
pd_trc='PD_FINAL_BASE'
tipo_banda = 'Jeffrey'#Jeffrey Vasiseck
codmes_default = 202307
columna_monto='MTOAPROBADO_SOLES'
titulo='Calibración Applicant App Ahorro Local'
condiciones= "CODMES >= 201901 AND DEF_120_BLOQ_REF_0 = 0"

In [0]:
# Objeto de Seguimiento
Obj_Seg = rmm.MonitorScore(
  nombre_modelo         = 'Personas Modelo Applicant Ahorro Local',
  codigo_modelo         = 'MOD-BCP-20658',
  tier                  = 'II',
  detalle_seguimiento   = 'Seguimiento Applicant Ahorro Local',
  mes_seguimiento       = '202504',
  base_monitoreo        = base_seguimiento,
  pd1                   = pd_cal,
  pd2                   = pd_cal,
  monto_credito         = columna_monto,
  query_univ            = "DEF_120_BLOQ_REF_0=0 AND PD_2024Q4vf IS NOT NULL AND FLG_VC_VIVIENDA_RBM=1",
  bandas                = tipo_banda,
  codmes_default        = codmes_default,
  meses_ventana_target  = 24,
  meses_atraso_target   = 4
)

In [0]:
dis_veh_app, evoldis_gini_pd_cal, evoldis_ks_pd_cal = Obj_Seg.EvolDis(
  query_filtro       = 'FLAG_CLIENTE=1',
  pd_dis             = 'XB_OPE_CLI_CAL_REP',
  codmes_ini         = 202206,
  codmes_fin         = 202409,
  amplitud           = 6,
  disc_target        = [24, 21,18],
  umbral_sup         = 0.4,
  umbral_inf         = 0.3,
  moviles            = True,
  dim_grafico        = (22.5, 6),
  pos_leyenda        = (0.5, -0.3),
  etiquetas          = True,
  eje_gini_max = 1,
  eje_gini_min = 0
)

In [0]:
dis_veh_app, evoldis_gini_pd_cal, evoldis_ks_pd_cal = Obj_Seg.EvolDis(
  query_filtro       = 'FLAG_CLIENTE=0',
  pd_dis             = 'XB_OPE_NOCLI_REP',
  codmes_ini         = 202101,
  codmes_fin         = 202409,
  amplitud           = 6,
  disc_target        = [24, 21,18],
  umbral_sup         = 0.4,
  umbral_inf         = 0.3,
  moviles            = True,
  dim_grafico        = (22.5, 6),
  pos_leyenda        = (0.5, -0.3),
  etiquetas          = True,
  eje_gini_max = 1
)

In [0]:
var_pd = ["XB_FINAL_BASE_REP", "XB_FINAL_BASE"]
caldri_veh_app = Obj_Seg.EvolDisVar(
  # Parametros de figura individual
  query_filtro      = 'FLAG_CLIENTE =1',
  codmes_ini        = 202206,
  codmes_fin        = 202312,
  amplitud          = 6,
  disc_target       = [24, 21,18],
  umbral_sup        = 0.4,
  umbral_inf        = 0.3,
  moviles           = True,
  dim_grafico       = (22.5, 6),
  pos_leyenda       = (0.5, -0.35),
  etiquetas         = True,
  gini_abs          = False,
 
  # Parametros de figura final
  ncolumnas         = 2,
  nfilas            = 1,
  variables         = var_pd,
  dim_grafico_total = (20, 6),
  ncol_leyenda      = 5,
  vspacio           = -0.25,
  hspacio           = 4,
  tamaño_etiqueta   = 14 
)

In [0]:
var_pd = ["XB_FINAL_BASE_REP", "XB_FINAL_BASE"]
caldri_veh_app = Obj_Seg.EvolDisVar(
  # Parametros de figura individual
  query_filtro      = 'FLAG_CLIENTE =0',
  codmes_ini        = 202206,
  codmes_fin        = 202312,
  amplitud          = 6,
  disc_target       = [24, 21,18],
  umbral_sup        = 0.4,
  umbral_inf        = 0.3,
  moviles           = True,
  dim_grafico       = (22.5, 6),
  pos_leyenda       = (0.5, -0.35),
  etiquetas         = True,
  gini_abs          = False,
 
  # Parametros de figura final
  ncolumnas         = 2,
  nfilas            = 1,
  variables         = var_pd,
  dim_grafico_total = (20, 6),
  ncol_leyenda      = 5,
  vspacio           = -0.25,
  hspacio           = 4,
  tamaño_etiqueta   = 14 
)

In [0]:
var_pd = ["PD_P1_REP", "PD_P1"]
caldri_veh_app = Obj_Seg.EvolDisVar(
  # Parametros de figura individual
  query_filtro      = '',
  codmes_ini        = 202206,
  codmes_fin        = 202312,
  amplitud          = 6,
  disc_target       = [24, 21,18],
  umbral_sup        = 0.4,
  umbral_inf        = 0.3,
  moviles           = True,
  dim_grafico       = (22.5, 6),
  pos_leyenda       = (0.5, -0.35),
  etiquetas         = True,
  gini_abs          = False,
 
  # Parametros de figura final
  ncolumnas         = 2,
  nfilas            = 1,
  variables         = var_pd,
  dim_grafico_total = (20, 6),
  ncol_leyenda      = 5,
  vspacio           = -0.25,
  hspacio           = 4,
  tamaño_etiqueta   = 14 
)

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "SUITE_AntLaboralMeses_J", 
    when(col("codmes") <= 202307, col("ANT_LABORAL_MESES")).otherwise(col("SUITE_AntLaboralMeses"))
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "SUITE_AntLaboralMeses_J",
    col('SUITE_AntLaboralMeses_J').cast('int')
)

# Cortamos la variable
plazo = Obj_Seg.CutVar(variables=['EDAD', 'APALANCAMIENTO', 'SUITE_AntLaboralMeses_J'], num_cortes=4, query_corte="CODMES BETWEEN 202101 AND 202410 AND DEF_120_BLOQ_REF_0 = 0")

In [0]:
var_cal = ["LTV_DEFINITIVA_Q", "FLG_ZONA_RIES_REP", "APALANCAMIENTO_Q", "EDAD_Q", "CTDPLAZOAPROBADO_N_REP_0", "TIPSITUACIONLABORAL_N"]

psi_dri = Obj_Seg.MixPSIVar(
  # Parametros de figura individual
    query_uso         ='FLAG_CLIENTE=1 AND CODMES >=202201',
    cast_int          =True,
    codmes_inicio1    =202011,
    codmes_fin1       =202204,
    codmes_inicio2    =202501,
    codmes_fin2       =202504,
    dim_grafico       =(25, 6),
    pos_leyenda       =(0.5, -0.35),
 
    # Parametros de figura final
    tipo_vista        = 'M',
    ncolumnas         = 3,
    nfilas            = 2,
    variables         = var_cal,
    dim_grafico_total = (27, 12),
    ncol_leyenda      =5,
    tamaño_leyenda    = 8.25,
    vspacio           =-0.45,
    hspacio           =0,
    borde             = 0.08
)

In [0]:
var_cal = ["LTV_DEFINITIVA_Q", "FLG_ZONA_RIES_REP", "APALANCAMIENTO_Q", "EDAD_Q", "CTDPLAZOAPROBADO_N_REP_1", "SUITE_AntLaboralMeses_J_Q"]

psi_dri = Obj_Seg.MixPSIVar(
  # Parametros de figura individual
    query_uso         ='FLAG_CLIENTE=0 AND CODMES >=202201',
    cast_int          =True,
    codmes_inicio1    =202011,
    codmes_fin1       =202204,
    codmes_inicio2    =202501,
    codmes_fin2       =202504,
    dim_grafico       =(25, 6),
    pos_leyenda       =(0.5, -0.35),
 
    # Parametros de figura final
    tipo_vista        = 'M',
    ncolumnas         = 3,
    nfilas            = 2,
    variables         = var_cal,
    dim_grafico_total = (27, 12),
    ncol_leyenda      =5,
    tamaño_leyenda    = 8.25,
    vspacio           =-0.45,
    hspacio           =0,
    borde             = 0.08
)

In [0]:
caldri_veh_app, rc_dri, rm_dri = Obj_Seg.CalDri(
  driver             = 'LTV_DEFINITIVA_Q',
  query_uso          = f"CODMES >= 202110 AND CODMES <= 202306 AND CODMES NOT IN (202304)",
  pd_seg             = 'PD_2024Q4vf',
  rd_aux             = 'RD12',
  pd_comparacion     = False,
  factores_cuentas   = fc,
  factores_montos    = fm,
  mat_importancia    = 5,
  cast_int           = True, 
  proys_def_temp     = 3,
  titulo             = f'LTV_DEFINITIVA_Q',
  dim_grafico        = (35, 6.5),
  etiquetas          = True,
  pos_etiquetas_xc   = [10, 10, 10, 10, 10, 10, 10, 10, 10],
  pos_etiquetas_xm   = [10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
  pos_leyenda        = (0.5, -0.2),
  punt_mora          = 100,
  iv = True
)